In [ ]:
movies = df2[['id', 'title']].copy()
movies.head(5)

In [ ]:
import pickle

In [ ]:
get_recommendations('The Dark Knight Rises', cosine_sim2)

In [ ]:
df2 = df2.reset_index()
indices = pd.Series(df2.index, index=df2['title'])
indices

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_sim2 = cosine_similarity(count_matrix, count_matrix)
cosine_sim2 

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer(stop_words = 'english')
count_matrix = count.fit_transform(df2['soup'])
count_matrix

In [ ]:
def create_soup(x):
    return ' '.join(x['keywords']) + ' ' + ' '.join(x['cast_y']) + ' ' + x['director'] + ' ' + ' '.join(x['genres'])
df2['soup'] = df2.apply(create_soup, axis=1)
df2['soup']

In [ ]:
df2[['title', 'cast_y', 'director', 'keywords', 'genres']].head(3)

In [ ]:
features = ['cast_y', 'keywords', 'director', 'genres']
for feature in features:
    df2[feature] = df2[feature].apply(clean_data)

In [ ]:
# 모든 문자를 소문자로 하고 띄어쓰기 없애기
def clean_data(x):
    if isinstance(x, list):
        return [str.lower(i.replace(' ','')) for i in x]
    else:
        if isinstance(x, str):
            return str.lower(x.replace(' ',''))
        else:
            return ''

In [ ]:
df2[['title', 'cast_y', 'director', 'keywords', 'genres']]  

In [ ]:
features = ['cast_y', 'keywords', 'genres']
for feature in features:
    df2[feature] = df2[feature].apply(get_list)

In [ ]:
# 처음 3개의 데이터 중 name 에 해당하는 데이터만 추출
def get_list(x):
    if isinstance(x, list):
        names = [i['name'] for i in x]
        if len(names) > 3:
            names = names[:3]
        return names
    return []

In [ ]:
df2['director'] = df2['crew_y'].apply(get_director)
df2['director']

In [ ]:
# 감독 정보 추출
def get_director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return np.nan

In [ ]:
df2.loc[0,'crew_y']

In [ ]:
from ast import literal_eval
features = ['cast_y', 'crew_y', 'keywords', 'genres']
for feature in features:
    df2[feature] = df2[feature].apply(literal_eval)

In [ ]:
# 인덱스 정보를 통해 영화 제목 추출
df2['title'].iloc[test_movie_indices]

In [ ]:
# 추천 영화 목록 10개의 인덱스 정보 추출
test_movie_indices = [i[0] for i in test_sim_scores[1:11]]
test_movie_indices

In [ ]:
test_sim_scores = sorted(test_sim_scores, key = lambda x : x[1], reverse = True)
test_sim_scores[1:11] # 자기 자신을 제외한 10개의 추천 영화를 슬라이싱

In [ ]:
def get_recommendations(title, cosine_sim= cosine_sim):
    # 영화 제목을 통해서 전체 데이터 기준 그 영화의 index 값을 얻기
    idx = indices[title]

    # 코사이 유사도 매트릭스에서 idx에 해당하는 데이터를 idx, 유사도 형태로 얻어기
    sim_scores = list(enumerate(cosine_sim[idx]))

    # 코사인 유사도 기준으로 내림차순 정렬
    sim_scores = sorted(sim_scores, key = lambda x : x[1], reverse = True)
    
    # 자기 자신을 제외한 10개의 추천 영화를 슬라이싱
    sim_scores = sim_scores[1:11]

    # 추천 영화 목록 10개의 인덱스 정보 추출
    movie_indices = [i[0] for i in sim_scores]

    # 인덱스 정보를 통해 영화 제목 추출
    return df2['title'].iloc[movie_indices]
    

In [ ]:
indices = pd.Series(df2.index, index = df2['title']).drop_duplicates()
indices

In [ ]:
cosine_sim.shape

In [ ]:
from sklearn.metrics.pairwise import linear_kernel

cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix) # 코사인 유사도 데이터
cosine_sim

In [ ]:
tfidf_matrix = tfidf.fit_transform(df2['overview'])
tfidf_matrix.shape # 20978 의 단어가 재재

In [ ]:
df2['overview'] = df2['overview'].fillna('') # null 에 빈 문자열이라도 채워준

In [ ]:
df2['overview'].isnull().values.any() # overview 값에 null이 있는지 판단

In [ ]:
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
ENGLISH_STOP_WORDS

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(stop_words = 'english')

In [ ]:
df2['overview'].head(5)

In [ ]:
q_movies = q_movies.sort_values('score', ascending = False)
q_movies[['title', 'vote_count', 'vote_average', 'score']].head(10)

In [ ]:
q_movies['score'] = q_movies.apply(weighted_rating, axis = 1)
q_movies.head(3)

In [ ]:
def weighted_rating(x, m =m, C = C):
    v = x['vote_count']
    R = x['vote_average']
    return (v/ (v+m) * R) + ( m/ (m+v) * C)

In [ ]:
q_movies['vote_count'].sort_values()

In [ ]:
q_movies = df2.copy().loc[df2['vote_count'] >= m]
q_movies.shape

In [ ]:
m = df2['vote_count'].quantile(0.9) #상위 10퍼센트에 대한 데이터를 뽑아줌
m # 1838개의 평가수가 상위 10퍼센트 평가수이다

In [ ]:
C = df2['vote_average'].mean()
C

In [ ]:
df2 = df2.merge(df1[['id', 'cast', 'crew']], on = 'id') # id 에 맞춰 뒤에 add를 해준다
df2.head(3)

In [ ]:
df1[['id', 'cast', 'crew']]

In [ ]:
df1.columns = ['id', 'title', 'cast', 'crew']

df1.shape, df2.shape

In [ ]:
df2.head(3)

In [ ]:
df1.head()

In [ ]:
import pandas as pd
import numpy as np

df1 = pd.read_csv('tmdb_5000_credits.csv')
df2 = pd.read_csv('tmdb_5000_movies.csv')